# Predicting house prices

data - Boston Housing price dataset

- regression

In [1]:
#importing data and loading

from keras.datasets import boston_housing
(train_data, train_target),(test_data, test_target) = boston_housing.load_data()

Using TensorFlow backend.


57344/57026 [==============================] - 1s 11us/step


In [2]:
train_data.shape, test_data.shape

((404, 13), (102, 13))

In [4]:
train_data[0]

array([  1.23247,   0.     ,   8.14   ,   0.     ,   0.538  ,   6.142  ,
        91.7    ,   3.9769 ,   4.     , 307.     ,  21.     , 396.9    ,
        18.72   ])

In [5]:
train_target[0]

15.2

15,200 dollars

### data preperation

In [7]:
#normalizing data
mean = train_data.mean(axis=0)
train_data=train_data-mean
std=train_data.std(axis=0)
train_data=train_data/std

In [8]:
test_data=test_data-mean
test_data=test_data/std

### network

In [13]:
from keras import layers,models

def build_model():
    model=models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop',loss='mse',metrics=['mae'])
    return model

k-fold validation

In [14]:
import numpy as np

In [15]:
k=4
num_val_samples = len(train_data)//k
num_epochs=100
all_scores=[]

In [18]:
for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i*num_val_samples : (i+1)*num_val_samples]
    val_target = train_target[i*num_val_samples : (i+1)*num_val_samples]
    
    x_train = np.concatenate([train_data[:i*num_val_samples], train_data[(i+1)*num_val_samples:]], axis=0)
    y_train = np.concatenate([train_target[:i*num_val_samples], train_target[(i+1)*num_val_samples:]], axis=0)
    
    model=build_model()
    model.fit(x_train,y_train,epochs=num_epochs,batch_size=1,verbose=0)
    
    val_mse, val_mae = model.evaluate(val_data,val_target,verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [19]:
all_scores

[2.589756727218628, 2.219867706298828, 2.753755569458008, 2.4008357524871826]

In [20]:
np.mean(all_scores)

2.4910539388656616

we are nearly $2500 off

In [22]:
preds=model.predict(test_data)

In [23]:
preds[0]

array([11.650585], dtype=float32)

predicted cost of first test sample is $11,650